# AOC 2025, day 10
## A linear algebra solution to part 2

This is a write-up of the math that I used for my solution [aoc10c.py](aoc10c.py) which, as you might
notice by the name is the third python-language solution I wrote up for that day's problem. This solution
is perhaps six or seven times slower than my solution based on the divide-and-conquer insight that's
been widely shared, but I think that re-learning the linear algebra math was good for me, and might
possibly be good for someone else, so here's a write-up.

I'm going to walk through the linear algebra on one machine to show how it works; hopefully that should
make it quite possible to read the python solution code. If you're unfamiliar with the use of the
`numpy` library in python, the only bit of syntax you really need to know to usnderstand this is that
in `numpy` the `@` symbol is used to mean "matrix multiplication". Other than that, it should be fairly
straightforward assuming that you're familiar with the problem.

Because this is an interactive python notebook where I have to actually do all the work, first some
imports:

In [1]:
import aoc10c as aoc10
import pathlib
import re
import numpy as np

The only function I'm going to use from `aoc10` is the function `row_reduce`, which
takes a matrix of integers to the closest you can get to "[reduced row-echelon form](https://en.wikipedia.org/wiki/Row_echelon_form#Reduced_row_echelon_form)"
while guaranteeing that you stay all in integers. (Namely, the pivot elements might
not all be $1$, because doing that might require making some other elements of the
row non-integer)

I think that how that function works is fairly self-evident from the code, but let
me know if I need a separate explanation of what that function does and how it does it.

In [2]:
np.set_printoptions(linewidth=200)

Now I go and parse the input; this isn't that interesting if you came here for the linear algebra, but necessary to get our machine.

In [3]:
datalines = pathlib.Path("aoc10.in").read_text().splitlines()
parser = re.compile(r"\[([.#]*)\] *((?:\([\d,]+\) *)+) *\{([\d,]+)\} *")
machines = []
for line in datalines:
    m = parser.fullmatch(line)
    assert m is not None
    pat = m.group(1)
    patnum = int("".join(reversed(pat.replace(".", "0").replace("#", "1"))), 2)
    button_nums = []
    raw_buttons = []
    for button in m.group(2).replace("(", "").split(")"):
        if button.strip():
            lnums = [int(x) for x in button.strip().split(",")]
            raw_buttons.append(tuple(lnums))
            button_nums.append(sum(2**x for x in lnums))
    joltages = [int(x) for x in m.group(3).split(",")]
    machines.append(
        {
            "pat": pat,
            "target": patnum,
            "buttons": button_nums,
            "jbuttons": raw_buttons,
            "jolts": joltages,
        }
    )


In [4]:
machines[1]

{'pat': '######..',
 'target': 63,
 'buttons': [63, 177, 162, 88, 196, 161, 156, 224, 122, 26],
 'jbuttons': [(0, 1, 2, 3, 4, 5),
  (0, 4, 5, 7),
  (1, 5, 7),
  (3, 4, 6),
  (2, 6, 7),
  (0, 5, 7),
  (2, 3, 4, 7),
  (5, 6, 7),
  (1, 3, 4, 5, 6),
  (1, 3, 4)],
 'jolts': [31, 51, 38, 61, 77, 74, 49, 72]}

So this is clearly a machine where we're underspecified: 10 buttons raising 8 joltages.

If we were to represent this as a set of linear equations in ten variables, we have:

$$
\begin{array}
\, b_0 & +b_1 &       &      &      & + b_5 & & & & &=& 31 \\
\, b_0 &      & + b_2 &      &      & & & & +b_8 & +b_9 &=& 51 \\
b_0 &      &       &      & +b_4 & & +b_6 & & & &=& 38 \\
b_0 &      &       & +b_3 & & & +b_6 & & +b_8 & +b_9 &=& 61 \\
b_0 & +b_1 &   & +b_3 & & & +b_6 & & +b_8 & +b_9 &=& 77 \\
b_0 & +b_1 & +b_2 &  & & +b_5 &  & +b_7 & +b_8 & &=& 74 \\
 & & & b_3 & +b_4 & &  & +b_7 & +b_8 & &=& 49 \\
 & b_1 & +b_2 & & +b_4 & +b_5 & +b_6  & +b_7 & & &=& 72 \\ 
\end{array}
$$

So first let's try to get those as matrices:

In [5]:
jolts = machines[1]["jolts"]
buttons = machines[1]["jbuttons"]

jolt_col_vec = np.array([list(jolts)], dtype=int).transpose()
button_mat = np.zeros((len(jolts), len(buttons)), dtype=int)
for button_idx, button in enumerate(buttons):
    button_mat[tuple(button), button_idx] = 1

print(button_mat)

[[1 1 0 0 0 1 0 0 0 0]
 [1 0 1 0 0 0 0 0 1 1]
 [1 0 0 0 1 0 1 0 0 0]
 [1 0 0 1 0 0 1 0 1 1]
 [1 1 0 1 0 0 1 0 1 1]
 [1 1 1 0 0 1 0 1 1 0]
 [0 0 0 1 1 0 0 1 1 0]
 [0 1 1 0 1 1 1 1 0 0]]


Okay, so the problem we are trying to solve is to find a column vector of ten non-negative
integers so that `button_mat @ bvect == jolt_col_vec`, with the sum as small as possible.

Now there may be a more clever way to solve this, but like many people I chose to add some extra
fake joltage values that were tied to only one button each and then use well-known techniques
that work on solving an equation with an equal number of unknowns and equations, and vary the
fake values through all their possibilities, looking only at the values that result in a
solution with non-negative integers.

For now, I'm going to add my two fake joltages as extra outputs of the last two buttons (button
8 and button 9):

$$
\begin{array}
\, b_0 & +b_1 &       &      &      & + b_5 & & & & &=& 31 & &\\
\, b_0 &      & + b_2 &      &      & & & & +b_8 & +b_9 &=& 51 & & \\
b_0 &      &       &      & +b_4 & & +b_6 & & & &=& 38 & & \\
b_0 &      &       & +b_3 & & & +b_6 & & +b_8 & +b_9 &=& 61 & & \\
b_0 & +b_1 &   & +b_3 & & & +b_6 & & +b_8 & +b_9 &=& 77 & & \\
b_0 & +b_1 & +b_2 &  & & +b_5 &  & +b_7 & +b_8 & &=& 74 & & \\
 & & & b_3 & +b_4 & &  & +b_7 & +b_8 & &=& 49 & & \\
 & b_1 & +b_2 & & +b_4 & +b_5 & +b_6  & +b_7 & & &=& 72 & & \\ 
\, & & & & & &   &  & b_8 & &=& &f_1& \\ 
\, & & & & & &   &  & & b_9 &=& & &f_2 \\ 
\end{array}
$$

Where I'm using $f_1$ and $f_2$ as the fake joltage values. We can extend the button matrix to
add these two extra rows and represent the fake joltage values using a 3-column matrix:

In [6]:
extended_button_mat = np.copy(button_mat)
extended_button_mat.resize((extended_button_mat.shape[0]+2, extended_button_mat.shape[1]))
extended_button_mat[-2,8] = extended_button_mat[-1,9] = 1

extended_joltages = np.zeros((len(jolts)+2, 3), dtype=int)
extended_joltages[0:len(jolts),0:1] = jolt_col_vec
extended_joltages[-2,-2] = extended_joltages[-1,-1] = 1

In [7]:
print(extended_button_mat)

[[1 1 0 0 0 1 0 0 0 0]
 [1 0 1 0 0 0 0 0 1 1]
 [1 0 0 0 1 0 1 0 0 0]
 [1 0 0 1 0 0 1 0 1 1]
 [1 1 0 1 0 0 1 0 1 1]
 [1 1 1 0 0 1 0 1 1 0]
 [0 0 0 1 1 0 0 1 1 0]
 [0 1 1 0 1 1 1 1 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 1]]


In [8]:
print(extended_joltages)

[[31  0  0]
 [51  0  0]
 [38  0  0]
 [61  0  0]
 [77  0  0]
 [74  0  0]
 [49  0  0]
 [72  0  0]
 [ 0  1  0]
 [ 0  0  1]]


So now we're going to find a column vector of ten non-negative
integers so that `extended_button_mat @ bvect == extended_joltages @ [1, f1, f2]`,
with the sum as small as possible. But since `extended_button_mat` is a
square matrix, there should be (absent unlucky things like duplicate rows)
only one solution for any pair of values for `f1` and `f2`.

The "just use the machinery" way at this point would be to invert the matrix
`extended_button_mat` and then begin checking every possible `f1` and `f2`
value with `extended_button_mat_inverse @ extended_joltages @ [1, f1, f2]`, tracking
the sum when we had all positive integer answers. But let's see if we can do this
without using floating point math and sticking just to integer operations.

Let's instead see what happens when we *row reduce* the matrix formed by slapping
`extended_button_mat` and `extended_joltages` together:

In [9]:
combined = np.hstack((extended_button_mat, extended_joltages))
print("Before row_reduce"); print(combined)
aoc10.row_reduce(combined)
print("After row_reduce"); print(combined)

Before row_reduce
[[ 1  1  0  0  0  1  0  0  0  0 31  0  0]
 [ 1  0  1  0  0  0  0  0  1  1 51  0  0]
 [ 1  0  0  0  1  0  1  0  0  0 38  0  0]
 [ 1  0  0  1  0  0  1  0  1  1 61  0  0]
 [ 1  1  0  1  0  0  1  0  1  1 77  0  0]
 [ 1  1  1  0  0  1  0  1  1  0 74  0  0]
 [ 0  0  0  1  1  0  0  1  1  0 49  0  0]
 [ 0  1  1  0  1  1  1  1  0  0 72  0  0]
 [ 0  0  0  0  0  0  0  0  1  0  0  1  0]
 [ 0  0  0  0  0  0  0  0  0  1  0  0  1]]
After row_reduce
[[  2   0   0   0   0   0   0   0   0   0  40  -1   0]
 [  0   1   0   0   0   0   0   0   0   0  16   0   0]
 [  0   0   2   0   0   0   0   0   0   0  62  -1  -2]
 [  0   0   0   4   0   0   0   0   0   0 120  -3  -4]
 [  0   0   0   0   4   0   0   0   0   0  28   1   0]
 [  0   0   0   0   0   2   0   0   0   0 -10   1   0]
 [  0   0   0   0   0   0   4   0   0   0  44   1   0]
 [  0   0   0   0   0   0   0   2   0   0  24  -1   2]
 [  0   0   0   0   0   0   0   0   1   0   0   1   0]
 [  0   0   0   0   0   0   0   0   0   1   0   0

Look at that matrix post-row-reduce: a left hand side that's got stuff only on the diagonal, and nothing
elsewhere. Note also from this that it looks like we can read off the matrix that $b_1 = 16$, no matter
what we set for the fake joltage values. This isn't too terribly much of a surprise, since if you go
back and look at the original equations, you may notice these two:

$$
\begin{array}
\, b_0 &      &       & +b_3 & & & +b_6 & & +b_8 & +b_9 &=& 61 & & \\
b_0 & +b_1 &   & +b_3 & & & +b_6 & & +b_8 & +b_9 &=& 77 & & \\
\end{array}
$$

from which it is obvious that $b_1$ must be $77 - 61 = 16$.

Anyway, now we just go over all the possible values for our fake joltages - since the first fake joltage $f_1$
is connected to button $8$, we know that it can't be more than $49$ jolts (because if it were, we'd have to press
button $8$ so many times that we'd exceed joltage $\#6$), and likewise the second fake joltage $f_2$ can't be more
than $51$ jolts.

From the matrix we could read off further restrictions; for example, we also know that $f_1 \leq 40$ (since
otherwise our computed value for $b_0$ is negative) and furthermore that $f_1$ is a multiple of $4$ (so that our
computed value of $b_3$ is an integer), but unfortunately I wasn't able to code up that human insight in general,
so my full solution just tries every value from 0 through the maximum determined straight from the initial joltages.

To try a pair of fake joltage values $f_1$ and $f_2$ what we do is make up a (column) vector of
$[ 1 ; f_1 ; f_2 ]$ and take the matrix product of the three right-most rows of the matrix we found with
`row_reduce` above and that column vector, and that should give us almost our values for button presses. We
check that all our values are non-negative and that they're all multiples of the value on the diagonal
in the left hand side of that reduced matrix.

In [10]:
square_part, mul_part = np.hsplit(combined, [len(buttons)])
print(square_part)
print(mul_part)

[[2 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 2 0 0 0 0 0 0 0]
 [0 0 0 4 0 0 0 0 0 0]
 [0 0 0 0 4 0 0 0 0 0]
 [0 0 0 0 0 2 0 0 0 0]
 [0 0 0 0 0 0 4 0 0 0]
 [0 0 0 0 0 0 0 2 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 1]]
[[ 40  -1   0]
 [ 16   0   0]
 [ 62  -1  -2]
 [120  -3  -4]
 [ 28   1   0]
 [-10   1   0]
 [ 44   1   0]
 [ 24  -1   2]
 [  0   1   0]
 [  0   0   1]]


In [11]:
diag_values = square_part.sum(axis=1)
print(diag_values)

[2 1 2 4 4 2 4 2 1 1]


In [12]:
minsum = 1_000_000
minnib = None
minsol = None
for f1 in range(49+1):
    for f2 in range(51+1):
        testvec = mul_part @ np.array([1, f1, f2], dtype=int)
        if (testvec >= 0).all() and (testvec % diag_values == 0).all():
            # Valid solution!
            this_sum = (testvec // diag_values).sum()
            if this_sum < minsum:
                minnib = [1, f1, f2]
                minsol = np.copy(testvec // diag_values)
                minsum = this_sum
print(f"{minsum} = {minsol}.sum()")
print(minnib)

114 = [ 4 16 11  2 15 11 19  0 32  4].sum()
[1, 32, 4]


This logic, with a few variables renamed, is what I do in my solution [aoc10c.py](aoc10c.py).

The one tricky bit that I glossed over is attaching the fake joltages to buttons $b_8$ and $b_9$;
if I attached the fake joltages to other buttons, I might have run into issues. For example, if
I attached the fake joltages to buttons $b_0$ and $b_5$, I'd get:

In [13]:
extended_button_mat = np.copy(button_mat)
extended_button_mat.resize((extended_button_mat.shape[0]+2, extended_button_mat.shape[1]))
extended_button_mat[-2,0] = extended_button_mat[-1,5] = 1

combined = np.hstack((extended_button_mat, extended_joltages))
print("Before row_reduce"); print(combined)
aoc10.row_reduce(combined)
print("After row_reduce"); print(combined)

Before row_reduce
[[ 1  1  0  0  0  1  0  0  0  0 31  0  0]
 [ 1  0  1  0  0  0  0  0  1  1 51  0  0]
 [ 1  0  0  0  1  0  1  0  0  0 38  0  0]
 [ 1  0  0  1  0  0  1  0  1  1 61  0  0]
 [ 1  1  0  1  0  0  1  0  1  1 77  0  0]
 [ 1  1  1  0  0  1  0  1  1  0 74  0  0]
 [ 0  0  0  1  1  0  0  1  1  0 49  0  0]
 [ 0  1  1  0  1  1  1  1  0  0 72  0  0]
 [ 1  0  0  0  0  0  0  0  0  0  0  1  0]
 [ 0  0  0  0  0  1  0  0  0  0  0  0  1]]
After row_reduce
[[  1   0   0   0   0   0   0   0   0   0   0   1   0]
 [  0   1   0   0   0   0   0   0   0   0  16   0   0]
 [  0   0   1   0   0   0   0   0   0   1  11   1   0]
 [  0   0   0   2   0   0   0   0   0   2   0   3   0]
 [  0   0   0   0   2   0   0   0   0   0  34  -1   0]
 [  0   0   0   0   0   1   0   0   0   0  15  -1   0]
 [  0   0   0   0   0   0   2   0   0   0  42  -1   0]
 [  0   0   0   0   0   0   0   1   0  -1  -8   1   0]
 [  0   0   0   0   0   0   0   0   1   0  40  -2   0]
 [  0   0   0   0   0   0   0   0   0   0 -15   1

Note that last row there - it's telling me that there are no solutions at all unless my two fake
joltages add up to $15$, and even if the fake joltages do add up to $15$ I still won't have a
unique solution because I'll have to choose a value for $b_9$ to make everything work.
(note that the fourth-from-the-right column has non-zero values in multiple rows).

To avoid this situation, my solution code first does a `row_reduce` on what you get if you just
slap `button_vec` and `jolt_col_vec` together side-by-side, and then walks down the diagonal
to find missing buttons. If it finds any, it uses them and if not it then uses the last several
buttons. This ensures that I always attach the fake joltages such that my row-reduced matrix
doesn't have any zero rows in the left-hand-side part.